In [133]:
import sys
import random
import quandl
import gurobipy
import pandas as pd
import numpy as np

import dwave
import dimod
import dwave_qbsolv
import networkx as nx

from dimod import BinaryQuadraticModel
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite, EmbeddingComposite, VirtualGraphComposite
from helpers.draw import draw

from time import time
from math import sqrt
from pprint import pprint
from gurobipy import *
from itertools import combinations_with_replacement, product

from bokeh.plotting import figure, show, ColumnDataSource, save
from bokeh.models import Range1d, LabelSet, Row, HoverTool, WheelZoomTool, ZoomInTool, ZoomOutTool, PanTool
from bokeh.models.graphs import from_networkx
from bokeh.io import output_notebook, output_file


In [130]:

# Create a K4 complete graph (default node labels are indexical from 0)
G = nx.complete_graph(4)
# Randomly assign +1 or -1 relationship signs to all edges. Rename node 0 to Alice, 1 to Bob, etc
G.add_edges_from([(u, v) for u, v in G.edges])
nx.relabel_nodes(G, {0: 'Alice', 1: 'Bob', 2: 'Eve', 3: 'Wally'}, copy=False)



plot = figure(title="Networkx Integration Demonstration", x_range=(-5,5), y_range=(-5, 5),
              tools="", toolbar_location=None)

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)

show(plot)

## Assets50

In [158]:
returns = pd.read_csv('data/securities.csv', index_col=0)
returns_matrix = np.asmatrix(returns)

assets = pd.read_csv('data/assets50.csv')
assets = assets.pivot(index='asset1', columns='asset2')
assets_matrix = np.asmatrix(assets)

covariance_matrix = np.nan_to_num(assets['covariance'].values)
sampled_covariance_matrix = np.nan_to_num(assets['sampled_covariance'].values)
mean_returns_matrix = np.diag(returns['total_return'].values)


In [160]:
nodes = dict(enumerate(returns.index.to_series().values))

budget = 1000000
kappa = 100
theta = 0.85
weight = 1
A = 0.0001

money_market = (1 + 0.02) ** (1 / 365) - 1
return_level = (1 + 0.07) ** (1 / 365) - 1

ub = 5 # known from presolved problem
R = len(returns) # Number of risky assets

# Chang value of n to increase or decrease precision
n = 0
ub = ub + n

G = nx.complete_graph(R)
nx.relabel_nodes(G, nodes, copy=False)
G.add_edges_from([(u, v, {'weight': 12}) for u, v in G.edges])

tools = [WheelZoomTool(), ZoomInTool(), ZoomOutTool(), PanTool(), HoverTool()]
plot = figure(title="Networkx Integration Demonstration", x_range=(-3,3), y_range=(-3, 3))
plot.add_tools(*tools)
plot.toolbar.active_scroll = tools[0]

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)

show(plot)

problem_hamiltonian = covariance_matrix * (kappa / budget) ** 2 * (returns_matrix[:,0] * returns_matrix[:,0].T) * A
constraint_hamiltonian = np.zeros((R,R)) # (returns_matrix[:,0] * returns_matrix[:,0].T) * (kappa / budget) ** 2 * weight


for i in range(R):

    # 1
    for k in range(ub):
        for l in range(ub):
            constraint_hamiltonian[k, l] += (theta**2 * sampled_covariance_matrix[i,i] - returns['total_return'][i])**2 * 2**(k+l-2*n)

#     # 2
#     for k in range(ub):
#         mul = 4 * returnlevel * (theta**2 * sampledcovar[i,i] - returns[i,1]) * returns[i,1] * 2**(k-n)
#         lin[(i,k,i,k)].append(mul)

#     # 3
#     for j in range(N):
#         for k in range(ub):
#             for l in range(ub):
#                 mul = 2 * (theta**2 * sampledcovar[i,i] - returns[i,1]) * (theta**2 * sampledcovar[i,j] - returns[i,1]*returns[j,1]) * 2**(k+l-2*n)
#                 if i==j and k==l:
#                     lin[(i,k,j,l)].append(mul)
#                 else:
#                     qua[(i,k,j,l)].append(mul)

#     # 4
#     for j in range(N):
#         for k in range(ub):
#             mul = 4 * returnlevel * returns[i,1] * (theta**2 * sampledcovar[i,j] - returns[i,1]*returns[j,1]) * 2**(k-n)
#             lin[(j,k,j,k)].append(mul)

#     # 5
#     for j in range(N):
#         for k in range(N):
#             for l in range(ub):
#                 for m in range(ub):
#                     mul = (theta**2 * sampledcovar[i,j] - returns[i,1]*returns[j,1]) * (theta**2 * sampledcovar[i,k] - returns[i,1]*returns[k,1]) * 2**(l+m-2*n)
#                     if j==k and l==m:
#                         lin[(j,l,k,m)].append(mul)
#                     else:
#                         qua[(j,l,k,m)].append(mul)

constraint_hamiltonian


# constraint_hamiltonian += np.diag(-2 * (kappa / budget) * returns['price'] * weight)
# constraint_hamiltonian += ((theta ** 2 * assets['sampled_covariance'] - returns['total_return']) ** 2)
# constraint_hamiltonian += np.diag(np.diag(4 * return_level * (theta**2 * assets['sampled_covariance'] - returns['total_return']) * returns['total_return']))



array([[ 16705.48559313,  33410.97118627,  66821.94237253, ...,
             0.        ,      0.        ,      0.        ],
       [ 33410.97118627,  66821.94237253, 133643.88474506, ...,
             0.        ,      0.        ,      0.        ],
       [ 66821.94237253, 133643.88474506, 267287.76949012, ...,
             0.        ,      0.        ,      0.        ],
       ...,
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ],
       [     0.        ,      0.        ,      0.        , ...,
             0.        ,      0.        ,      0.        ]])

## Live Stock Data

In [41]:
tickers = [
    'AAPL', 'MSFT', 'WMT', 'ACB', 'GE', 'NIO', 'BAC', 'AMZN', 'CMCSA', 'EOAN', 'BMW',
    'IBM', 'RIG', 'CTXS', 'INTC', 'MCD', 'EBAY', 'AXAHF'
]

quandl.ApiConfig.api_key = 'keFUu5WbUqqG7kxqBqEp'
data = quandl.get_table('WIKI/PRICES', ticker=tickers, 
                        qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                        date = { 'gte': '2009-01-01', 'lte': '2016-12-31' }, 
                        paginate=True)

# create a new dataframe with 'date' column as index
new = data.set_index('date')

# use pandas pivot function to sort adj_close by tickers
data = new.pivot(columns='ticker')['adj_close']

# check the head of the output
print(data.tail())

growth_rates = data.pct_change()
covariance = growth_rates.cov()
returns = growth_rates.mean()

print(returns)
print(covariance)

covariance_matrix = covariance.values
mean_returns_matrix = np.diag(returns.values)


ticker            AAPL    AMZN        BAC      CMCSA       CTXS   EBAY  \
date                                                                     
2016-12-23  115.080808  760.59  22.346222  34.754708  72.655362  29.79   
2016-12-27  115.811668  771.40  22.356110  34.862932  72.759338  30.24   
2016-12-28  115.317843  772.13  22.079254  34.656323  72.047501  30.01   
2016-12-29  115.288214  765.15  21.752960  34.479229  71.999512  29.98   
2016-12-30  114.389454  749.87  21.851837  34.102904  71.431642  29.69   

ticker             GE         IBM       INTC         MCD       MSFT    RIG  \
date                                                                         
2016-12-23  31.055560  160.477796  35.749710  120.774186  61.864790  14.73   
2016-12-27  31.075043  160.891721  35.846409  120.705531  61.903920  15.22   
2016-12-28  30.880215  159.977235  35.420932  120.323024  61.620226  14.91   
2016-12-29  30.889956  160.371908  35.449942  120.430910  61.532183  14.71   
2016-12-30  3

In [20]:
np.fill_diagonal(covariance_matrix, 0)
qubo_matrix = covariance_matrix - mean_returns_matrix

# qubo_matrix

print(returns.index.columns)

AttributeError: 'Index' object has no attribute 'columns'

In [14]:

# Variance-covariance matrix
# covariance = np.matrix([
#     [+4, +3, -1,  +0],
#     [+3, +6, +1,  +0],
#     [-1, +1, +10, +0],
#     [+0, +0, +0,  +0]
# ])

# Returns
# expected_returns = np.array([8, 9, 12, 7])

# coefs = np.matrix([
# #    a   b   c   d
#     [+0, +3, +1, +4], # a
#     [+0, +0, +1, +2], # b
#     [+0, +0, +0, +6], # c
#     [+0, +0, +0, +0]  # d
# ])

# linear = {
#     0: -8,
#     1: -9,
#     2: -12,
#     3: -7
# }

# quadratic = {
#     (1, 0): 3,
#     (2, 0): -1, (2, 1): 1,
#     (3, 0): 4,  (3, 1): 2, (3, 2): 6
# }

bqm = BinaryQuadraticModel.from_numpy_matrix(qubo_matrix)
# bqm2 = dimod.BinaryQuadraticModel(linear, quadratic, 0.0, dimod.BINARY)

reads = 1000
sol_limit = 64

system = EmbeddingComposite(DWaveSampler(solver='DW_2000Q_2_1'))
sampler = dwave_qbsolv.QBSolv()

t_start = time()
# response = sampler.sample(bqm, num_reads=reads, solver_limit=sol_limit)
response = system.sample(bqm, num_reads=reads)
t_end = time()

print(response)

# BEST SOLUTION AVAILABLE RN
best_solution = response.first
print(response)

solution_data = list(dict(best_solution[0]).values())
solution_data = pd.DataFrame(solution_data, index=growth_rates.columns)
solution_data = pd.concat((mean_returns * 10000, solution_data), axis=1)
solution_data.columns = ['Weight', 'Mean_return']

Y
# solution_data

# # Objective value
# def objective_value(Y):
#     sum = 0
#     for i in range(M):
#         for j in range(M):
#             for k in range(N):
#                 for l in range(N):
#                     sum = sum + covariance[i,j] * 2**(l+k-2*n) * Y[i,k] * Y[j,l]
#     return sum

# def constraint_satisfied(Y):
#     # Funds and target
#     f_temp,t_temp = 0,0
#     for i in range(M):
#         for k in range(N):
#             f_temp = f_temp + 2**(k-n) * Y[i,k]
#             t_temp = t_temp + expected_returns[i] * 2**(k-n) * Y[i,k]
#     return f_temp, t_temp

# def get_solution(Y):
#     X = []
#     for i in range(M):
#         sum = 0
#         for k in range(N):
#             sum = sum + Y[i,k] * 2**(k-n)
#         X.append(sum)
#     return X

print("\nTime: ", t_end - t_start)
print("\nObjective: ", objective_value(Y))
used_funds, target_achieved = constraint_satisfied(Y)
# print("Original Funds: ", funds, " Funds used : ", used_funds)
# print("Target: ", target, " Achieved target : ", target_achieved)
# print("Solution = ", get_solution(Y))


     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 ... 48      energy num_oc. ...
0    1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ...  1 -452.109101       5 ...
45   1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ...  1 -452.109101       1 ...
73   1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ...  1 -452.109101       1 ...
108  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ...  1 -452.109101       1 ...
1    1  1  1  1  1  1  1  1  1  1  1  1  1  0  1 ...  1  -451.98203      33 ...
51   1  1  1  1  1  1  1  1  1  1  1  1  1  0  1 ...  1  -451.98203       1 ...
53   1  1  1  1  1  1  1  1  1  1  1  1  1  0  1 ...  1  -451.98203       1 ...
56   1  1  1  1  1  1  1  1  1  1  1  1  1  0  1 ...  1  -451.98203       1 ...
68   1  1  1  1  1  1  1  1  1  1  1  1  1  0  1 ...  1  -451.98203       1 ...
80   1  1  1  1  1  1  1  1  1  1  1  1  1  0  1 ...  1  -451.98203       1 ...
90   1  1  1  1  1  1  1  1  1  1  1  1  1  0  1 ...  1  -451.98203       1 ...
122  1  1  1  1  1  1  1  1  1  1  1  1 

NameError: name 'growth_rates' is not defined

In [25]:

response

SampleSet(rec.array([([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], -452.10910135,   5, 0.        ),
           ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], -451.98202975,  33, 0.        ),
           ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], -451.91923643,   3, 0.        ),
           ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], -451.86797062,  14, 0.        ),
           ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], -451.80517493,   1, 0.        ),
           ([1, 1, 1

In [ ]:
# !pip install -r requirements.txt

import sys
import dwave
import dimod
import dwave_qbsolv
import numpy as np
import pandas as pd

from dimod import (
    BinaryQuadraticModel
)
from dwave.system.composites import (
    FixedEmbeddingComposite,
    EmbeddingComposite,
    VirtualGraphComposite
)
from dwave.system.samplers import DWaveSampler

from pprint import pprint
from time import time

# Variance-covariance matrix
covariance = np.matrix([
    [+4, +3, -1,  +0],
    [+3, +6, +1,  +0],
    [-1, +1, +10, +0],
    [+0, +0, +0,  +0]
])

# Returns
expected_returns = [8, 9, 12, 7]
funds = 20
target = 100
# Securities
M = len(expected_returns)
# Upperbound of each investment
N = int(np.ceil(np.log2(funds)))

# Change value of n to increase or decrease precision
n = 4
N = N + n


coefs = np.matrix([
#    a   b   c   d
    [+4, +0, +0, +0], # a
    [+3, +6, +0, +0], # b
    [-1, +1, +9, +0], # c
    [+0, +0, +0, +0]  # d
])

# (cov(a,a))a + (cov(b,b))b + (cov(a,b) * 2^())a*b


def prob_hamiltonian():
    # Multiplier added as 0.0001
    lin = {}
    qua = {}
    # Objective function
    for i in range(M):
        for j in range(M):
            for k in range(N):
                for l in range(N):
                    mul = covariance[i, j] * 2 ** (l + k - 2 * n) * 0.0001
                    if i == j and k == l: # Diferrence between quadratic and linear
                        try:
                            lin[(i, k, j, l)].append(mul)
                        except KeyError:
                            lin[(i, k, j, l)] = [mul]
                    else:  
                        try:
                            qua[(i, k, j, l)].append(mul)
                        except KeyError:
                            qua[(i, k, j, l)] = [mul]
    return lin, qua

lin, qua = prob_hamiltonian() 

def constraint_hamiltonian(lin, qua):
    # Funds constraint
    weight1 = 1
    for i in range(M):
        for j in range(M):
            for k in range(N):
                for l in range(N):
                    if i == j and k == l: # Diferrence between quadratic and linear
                        lin[(i, k, j, l)].append(weight1 * 2 ** (k + l - 2 * n))
                    else:
                        qua[(i, k, j, l)].append(weight1 * 2 ** (k + l - 2 * n))
    for i in range(M):
        for k in range(N):
            lin[(i, k, i, k)].append(-weight1 * 2 * funds * 2 ** (k - n))
    
    # Target constraint
    for i in range(M):
        for j in range(M):
            for k in range(N):
                for l in range(N):
                    if i == j and k == l: # Diferrence between quadratic and linear
                        lin[(i, k, j, l)].append(expected_returns[i] * expected_returns[j] * 2 ** (k + l - 2 * n))
                    else:
                        qua[(i, k, j, l)].append(expected_returns[i] * expected_returns[j] * 2 ** (k + l - 2 * n))
    for i in range(M):
        for k in range(N):
            lin[(i, k, i, k)].append(-2 * target * expected_returns[i] * 2 ** (k - n))
    
    return lin, qua

lin, qua = constraint_hamiltonian(lin, qua)

def group_similar_terms(qua):
    # Group similar terms together eg. X17 * X34 = X34 * X17
    temp = {}
    while bool(qua):
        key = list(qua.keys())
        ref_key = key[0]
        grouped_key = ref_key[2:] + ref_key[:2]

        temp[ref_key] = qua[ref_key] + qua[grouped_key]
        del qua[ref_key]
        del qua[grouped_key]
    
    return temp

qua = group_similar_terms(qua)

def formalize(lin, qua):
    linear = {key[:2] : sum(lin[key]) for key in lin}

    quadratic = {(key[:2], key[2:]) : sum(qua[key]) for key in qua}

    return linear, quadratic

linear, quadratic = formalize(lin,qua)

def scale_bias_couplings(linear, quadratic):
    # IF coupling less than -0.8 : Some problem (check notes)
    
    linear_max = max(np.abs(list(linear.values())))
    quadratic_max = max(np.abs(list(quadratic.values())))

    scaling_factor = max(linear_max, quadratic_max)
    
    if min(list(quadratic.values())) / scaling_factor == -1:
        print("Warning: Check Coupling strengths")

    scaled_linear = {key : linear[key] / scaling_factor for key in linear}
    scaled_quadratic = {key : quadratic[key] / scaling_factor for key in quadratic}
    
    return scaled_linear, scaled_quadratic

scaled_linear, scaled_quadratic = scale_bias_couplings(linear, quadratic)

# bqm = dimod.BinaryQuadraticModel(scaled_linear, scaled_quadratic, 0.0, dimod.BINARY)

# reads = 1000
# sol_limit = 64

# system = EmbeddingComposite(DWaveSampler(solver='DW_2000Q_2_1'))
# sampler = dwave_qbsolv.QBSolv()

# Tref = time()
# response = sampler.sample(bqm, num_reads=reads, solver='tabu', solver_limit=sol_limit, verbosity=0)
# #response = system.sample(bqm, num_reads=reads)
# Tfin = time()

# # BEST SOLUTION AVAILABLE RN
# best_solution = response.first
# solution_data = list(dict(best_solution[0]).values())

# # Matrix form of solution
# Y = np.matrix([solution_data[N*i : N*i + N] for i in range(M)])

# # Objective value
# def objective_value(Y):
#     sum = 0
#     for i in range(M):
#         for j in range(M):
#             for k in range(N):
#                 for l in range(N):
#                     sum = sum + covariance[i,j] * 2**(l+k-2*n) * Y[i,k] * Y[j,l]
#     return sum

# def constraint_satisfied(Y):
#     # Funds and target
#     f_temp,t_temp = 0,0
#     for i in range(M):
#         for k in range(N):
#             f_temp = f_temp + 2**(k-n) * Y[i,k]
#             t_temp = t_temp + expected_returns[i] * 2**(k-n) * Y[i,k]
#     return f_temp, t_temp

# def get_solution(Y):
#     X = []
#     for i in range(M):
#         sum = 0
#         for k in range(N):
#             sum = sum + Y[i,k] * 2**(k-n)
#         X.append(sum)
#     return X

# print("\nTime : ", Tfin-Tref)
# print("Variables = Linear terms =", len(lin))
# print("Quadratic terms = ", len(qua))
# print("Objective : ", objective_value(Y))
# used_funds, target_achieved = constraint_satisfied(Y)
# print("Original Funds : ", funds, " Funds used : ", used_funds)
# print("Target : ", target, " Achieved target : ", target_achieved)
# print("Solution = ", get_solution(Y))


